In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import numpy as np
import os
import time
import collections
import math
import random
import zipfile
import reader
from matplotlib import pylab
from six.moves import range
from six.moves.urllib.request import urlretrieve
import csv
import jieba.posseg as jp, jieba

In [2]:
dir_name = 'dataset'
file_name = 'generate_select_distinct_name_zh_cn_temp.csv'
if not os.path.exists(dir_name+os.sep+file_name):
    file_name = urlretrieve(url + file_name, dir_name+os.sep+file_name)
else:
    print('文件 ',file_name, '已经存在.')

文件  select_description_from_sys_district.csv 已经存在.


In [3]:
document = open(dir_name+os.sep+file_name, 'rb').read().decode(encoding='utf-8')

# 文本长度是指文本中的字符个数
print ('Length of text: {} characters'.format(len(document)))

Length of text: 894832 characters


In [4]:
stop_file_name = 'stopwords.txt'
if os.path.exists(dir_name+os.sep+stop_file_name):
    stopWords = reader.read_data_as_array(stop_file_name)
else:
    quit()
stopword_list = set(stopWords)  # 停词。格式为{'', '她', '无', '至', '前后', '即令', '大', '往', '不特', '有的', '替代',。。。}

段落处理完毕


In [5]:
print(document[:255])

2019年开业60间房。上海时生隅园林文化酒店：2017年的秋末，时生隅初见。彼时的心愿，想要在快节奏的都市，寻一处“讲诉时光和生命的角落”。不问对错，不闻浮沉。这便是“时生隅”的由来。 这一次，我们秉承初心，在闹中取静的川沙地铁站商圈，觅得一方天地—充满江南气息的园林，叶园。长廊湖景，古雅庭院。这里淡雅而大气，是快节奏与民俗文化的交融，这里幽静而明快，是远道而来亦或小憩而居的别样天地。时生隅的新老朋友，我们在叶园，期待与您共襄当下景，把时光言欢。
2018年开业2018年装修143间房。云和夜泊酒店(上


In [6]:
flags = ('n', 'f', 's', 't', 'nr', 'ns', 'nt', 'nw', 'nz','v','vd','vn','a','ad','an','d','q','r','p','c','u','xc','PER','LOC','ORG')  # 词性

# cut_words = [w.word for w in jp.cut(document,use_paddle=True)]
cut_words = [w.word for w in jp.cut(document,use_paddle=True) if w.flag in flags and w.word not in stopWords and len(w.word) > 0]
# print(cut_words)
# for hotelDesc in document:
#     temp_words = [w.word for w in jp.cut(hotelDesc,use_paddle=True) if w.flag in flags and w.word not in stopWords and len(w.word) > 0]
#     temp_speechs = [w.flag for w in jp.cut(hotelDesc,use_paddle=True) if w.flag in flags and w.word not in stopWords and len(w.word) > 0]
# #     words = [w.word for w in jp.cut(text) if w.flag in flags and w.word not in stopwords and len(w.word) > 0]
# #     print('---{}, {}',temp_words, len(temp_words))
# #     print('---{}, {}',temp_speechs, len(temp_speechs))
#     if len(temp_words) > 0:
#         #[[句一分词列表],[句二分词列表],...[句五分词列表]]
#         words_ls.append(temp_words)
#         #[[句一词性列表],[句二词性列表],...[句五词性列表]]
#         words_speech.append(temp_speechs)
#         #五句合并的一个列表
#         whole_words.extend(temp_words)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.632 seconds.
Prefix dict has been built successfully.


In [7]:
vocab = sorted(set(cut_words))
print(vocab[:30])
print ('{} unique characters'.format(len(vocab)))

['4S店', '4s店', 'D座', 'IC卡', 'IP电话', '一个舒服', '一中', '一体', '一体机', '一分钱', '一刹那', '一及', '一号线', '一同', '一夕', '一大', '一定', '一客', '一家人', '一小', '一屋', '一川', '一带', '一幅', '一年四季', '一应', '一座座', '一新', '一族', '一景']
14509 unique characters


In [8]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in document])
print(text_as_int)

KeyError: '2'

In [ ]:
dict = {'\n': 0, ' ': 1, '!': 2, '"': 3, '%': 4}
dict['\n']

In [ ]:
print(idx2char)

In [ ]:
print(text_as_int)

In [ ]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

In [ ]:
# 设定每个输入句子长度的最大值
seq_length = 100
examples_per_epoch = len(text2)//seq_length

# 创建训练样本 / 目标
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

In [ ]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

In [ ]:
# 批大小
BATCH_SIZE = 128

# 设定缓冲区大小，以重新排列数据集
# （TF 数据被设计为可以处理可能是无限的序列，
# 所以它不会试图在内存中重新排列整个序列。相反，
# 它维持一个缓冲区，在缓冲区重新排列元素。） 
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

In [ ]:
# 词集的长度
vocab_size = len(vocab)

# 嵌入的维度
embedding_dim = 256

# RNN 的单元数量
rnn_units = 1024
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [ ]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

In [ ]:
model.summary()

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

In [ ]:
import time,datetime
model.compile(optimizer='adam', loss=loss)
# 检查点保存至的目录
checkpoint_dir = './training_checkpoints'

# 检查点的文件名
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_"+str(time.time()))

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True,
    save_freq=2)

In [ ]:
EPOCHS=1
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

In [ ]:
# latest = tf.train.latest_checkpoint(checkpoint_dir)
# model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

# model.load_weights(latest)
# model.build(tf.TensorShape([1, None]))
model.summary()

In [ ]:
def generate_text(model, start_string):
  # 评估步骤（用学习过的模型生成文本）

  # 要生成的字符个数
  num_generate = 10

  # 将起始字符串转换为数字（向量化）
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
  print(input_eval)
  # 空字符串用于存储结果
  text_generated = []

  # 低温度会生成更可预测的文本
  # 较高温度会生成更令人惊讶的文本
  # 可以通过试验以找到最好的设定
  temperature = 1.0

  # 这里批大小为 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      print(predictions)
      # 删除批次的维度
      predictions = tf.squeeze(predictions, 0)

      # 用分类分布预测模型返回的字符
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # 把预测字符和前面的隐藏状态一起传递给模型作为下一个输入
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))
print(generate_text(model, start_string=u"人民广场"))